# Data Generation Process
From HPC-ODA dataset we use: Cpu cycles, Instructions, Cache misses, Branch Instructions, Branch Misses and Power consumption. 

From PM100 (a dataset generated from the M100 dataset) we use: Cores Allocated, Memory Allocated, Gpu's Allocated, Time and Power Consumption.

Using relations between each of the HPC measures and the Power measure we built a percentace relation in order to then use those measure with the PM100 dataset.

### Upload HPC-ODA

In [1]:
### HPC_ODA
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import seaborn as sns 
import matplotlib.pyplot as plt


file_path = 'C://Users/Nuno/responses.csv'


power_data = pd.read_csv('C://Users/Nuno/HPC-ODA/power_prediction/responses/responses.csv', skiprows=1, usecols=[1], header=None, names=['y'])
cache_misses_data = pd.read_csv('C://Users/Nuno/HPC-ODA/power_prediction/sensors/node0.cache-misses.csv', skiprows=1, usecols=[1], header=None, names=['y'])
branch_instructions_data = pd.read_csv('C://Users/Nuno/HPC-ODA/power_prediction/sensors/node0.branch-instructions.csv', skiprows=1, usecols=[1], header=None, names=['y']) ##These are branch instructions.
branch_misses_data = pd.read_csv('C://Users/Nuno/HPC-ODA/power_prediction/sensors/node0.branch-misses.csv', skiprows=1, usecols=[1], header=None, names=['y']) ##These are branch instructions.
cpu_cycles_data = pd.read_csv('C://Users/Nuno/HPC-ODA/power_prediction/sensors/node0.cpu-cycles.csv', skiprows=1, usecols=[1], header=None, names=['y'])
instructions_data = pd.read_csv('C://Users/Nuno/HPC-ODA/power_prediction/sensors/node0.instructions.csv', skiprows=1, usecols=[1], header=None, names=['y']) ##These are instructions.


power = power_data['y'].values
cache_misses = cache_misses_data['y'].values
cache_misses = cache_misses[1:-1]
instructions = instructions_data['y'].values
instructions = instructions[1:-1]
cpu_cycles = cpu_cycles_data['y'].values
cpu_cycles = cpu_cycles[1:-1]
branch_instructions = branch_instructions_data['y'].values 
branch_instructions = branch_instructions[1:-1]
branch_misses = branch_misses_data['y'].values
branch_misses= branch_misses[1:-1]

instpow = np.divide(instructions,power)
cachepow = np.divide(cache_misses,power)
cpupow = np.divide(cpu_cycles,power)
brinspow = np.divide(branch_instructions,power)
brcachepow = np.divide(branch_misses,power)




### Upload PM100 (Completed only)

In [2]:
import seaborn as sns 
import matplotlib.pyplot as plt
import pandas as pd 

def load_data(path: str) -> pd.DataFrame:
    """Load the data from a parquet file.

    Args:
        path (str): Path to the parquet file.

    Returns:
        pd.DataFrame: Loaded DataFrame.
    """
    return pd.read_parquet(path)

if __name__ == "__main__":
    
    sns.set_theme(style="whitegrid")
    
    # Path to the PM100 data
    DATA_PATH = "C://Users/Nuno/job_table.parquet"
    
    # Load the dataframe
    df = load_data(DATA_PATH)
    
    # Filter out completed jobs
    df_completed = df[df['job_state'] == 'COMPLETED'].copy()  # Use copy() to explicitly create a copy
    
    # Exit state pie plot
    df_completed['job_state'].replace({"OUT_OF_MEMORY": "OOM+NODE FAIL", "NODE_FAIL": "OOM+NODE FAIL"}, inplace=True)

    # Plot the duration of the completed jobs divided by exit state
    # Convert runtime to minutes
    ##df_completed['run_time'] = df_completed['run_time'].apply(lambda rt: round(int(rt/60), -2))

    #df_completed['power_consumption'].to_csv('p1.csv', index=False)
    

In [3]:
def add_label (matched):
    i = 0
    for key in matched.keys():
        matched_list = matched[key]
        if len(matched_list) == 1:
            matched_list[0].append(0)
        elif len(matched_list) > 1:
            for i in range(len(matched_list)):
                matched_list[i].append(1)
                
    return (matched)

In [4]:
import random
import numpy as np

def aux(regular, irregular):
    high_bound = 0.05 * regular
    if high_bound < 2 :
        high_bound = 3
    random_values = []
    
    while sum(random_values) < regular:
        random_values.append(np.random.randint(2, high_bound))
        

    if sum(random_values) > regular:
        adjustment = sum(random_values) - regular
        while adjustment > 0:        
            max_index = random_values.index(max(random_values))
            random_values[max_index] -= 1
            adjustment -= 1
    
    array_of_ones = [1] * irregular
    
    random_values.extend(array_of_ones)
    return random_values



In [5]:
import numpy as np

def generate_tuples(l, noise, num, add=None):
    matched_list = l

    for i in range(num):
        lis = [0,0,0] 
        for j in range(len(lis)):
            n = noise[j]
            low_bound = matched_list[0][j] - matched_list[0][j] * n
            high_bound = matched_list[0][j] + matched_list[0][j] * n
            to_add = round(np.random.uniform(low_bound, high_bound))
            lis[j] = to_add
        matched_list.append(lis)
    
    return matched_list
            

In [6]:
def fit(l, dist, noise):
    matched_dic = l
    dist = sorted(dist, reverse=True)
    i = 0
    keys_to_change = []
    new_matched = []
    
    keys_to_delete = []
    for key in matched_dic.keys():
        matched_list = matched_dic[key]
        if i < len(dist):
            new = dist[i] - len(matched_list)
            if new > 0:
                matched_list = generate_tuples(matched_list, noise, new)

            elif new < 0:
                keys_to_change.append(key)
                matched_list = matched_list[:new]
                new_matched.append(matched_list)
            

            else :
                matched_list = matched_list

        
        elif i >= len(dist):
            keys_to_delete.append(key)
            
        
        i = i + 1
        
    j = 0
    for key in keys_to_change:
        matched_dic[key] = new_matched[j]
        j = j + 1

    for key_to_delete in keys_to_delete:
        del matched_dic[key_to_delete]
        

    return matched_dic


In [7]:
def update_percentage(l1,l2,l3,num_values, percentage, noise):      
    inst = l1
    bi = l2
    time = l3
    total_values = num_values
    p = percentage
    ni = noise[0] 
    nbi = noise[1] 
    nt = noise[2] 
    
    matched_inst, p_inst = check_percentage_individual(inst,total_values,ni)
    matched_bi, p_bi = check_percentage_individual(bi,total_values,nbi)
    matched_time,p_time = check_percentage_individual(time,total_values,nt)
    
    to_regular = round((percentage/100) * num_values)
    
    rest_to_regular = total_values - to_regular
    
    dist = aux(to_regular)

    
    inst, inst_2 = fit(matched_inst,dist,ni)
    bi, bi_2 = fit(matched_bi,dist,nbi)
    time, time_2 = fit(matched_time,dist,nt)

    if len(inst_2) >= rest_to_regular:
        excess = len(inst_2) - rest_to_regular
        inst_2 = inst_2[excess:]
    else:
        to_add = rest_to_regular - len(inst_2)
        to_fill = generate_instructions(to_add)
        inst_2 = np.concatenate((inst_2,to_fill))
    
    if len(bi_2) >= rest_to_regular:
        excess = len(bi_2) - rest_to_regular
        bi_2 = bi_2[excess:]
    else:
        to_add = rest_to_regular - len(bi_2)
        to_fill = generate_branch_instructions(to_add)
        bi_2 = np.concatenate((bi_2,to_fill))
    
    if len(time_2) >= rest_to_regular:
        excess = len(time_2) - rest_to_regular
        time_2 = time_2[excess:]
    else:
        to_add = rest_to_regular - len(time_2)
        to_fill = generate_time(to_add)
        time_2 = np.concatenate((time_2,to_fill))
        
    
    
    treated = list(zip(inst,bi,time))
    
    non_treated = list(zip(inst_2,bi_2,time_2))
    
    adjuction = treated + non_treated
    
    lim = len(dist)
    
    out = final_adjust(adjuction, total_values, noise, percentage, lim)
        
    

    return out


In [8]:
def final_adjust(l, num_values, noise, percentage, num):
    total_values = num_values
    ni = noise[0]
    nbi = noise[1]
    nt = noise[2]
    lim = num
    p = percentage
    check_tasks = l
    per_in = check_percentage(l,noise)
    regulated = round((percentage/100) * num_values)
    while per_in != p:
        i = 0 
        n1 = noise[0]
        n2 = noise[1]
        n3 = noise[2]
        
        for i in range(regulated , len(check_tasks)):
            value = check_tasks[i]
        
            for j in range(0, len(check_tasks)):  
                if (abs(check_tasks[j][0] - value[0]) <= n1 * value[0] and
                    abs(check_tasks[j][1] - value[1]) <= n2 * value[1] and
                    abs(check_tasks[j][2] - value[2]) <= n3 * value[2]) and (i != j):
                    arr1 = generate_instructions(total_values)
                    arr2 = generate_branch_instructions(total_values)
                    arr3 = generate_time(total_values)
                    random_inst = np.random.choice(arr1)
                    random_bi = np.random.choice(arr2)
                    random_time = np.random.choice(arr3)
                    check_tasks[i][0] = random_inst
                    check_tasks[i][1] = random_bi
                    check_tasks[i][2] = random_time
             
            per_in = check_percentage(check_tasks,noise)
            
            
    return check_tasks

    
        
    
    
    
    
    

In [9]:
def fix(matched_dic, total_values, interval, noise):
    matched = matched_dic
    lent = calc_values(matched)
        
    last_keys = list(matched.keys())[interval:]
    
    if lent > total_values:
        to_remove = lent - total_values
        keys_to_remove = random.sample(last_keys, to_remove)
        
        for key in keys_to_remove:
            del matched[key]
    else:
        while lent < total_values:
            to_add = total_values-lent
            matched = find_irr(matched,to_add,noise)
            
    return (matched)

In [10]:
def find_irr(matched,to_add,noise):
    instr = []
    bi = []
    tim = []
    for key in matched.keys():
        matched_list = matched[key]
        instr.append(matched_list[0][0])
        bi.append(matched_list[0][1])
        tim.append(matched_list[0][2])
    
    

In [30]:
import pandas as pd
import math
import sys


def generate_values(num_values,noise, percentage = None):
    total_values = (num_values)
    power = generate_power(total_values)
    inst = generate_instructions(total_values)
    cpucy = generate_cpu_cycles(total_values)
    cache_misses = generate_cache_misses(total_values)
    bi = generate_branch_instructions(total_values)
    bm = generate_branch_misses(total_values)
    memory = generate_memory(total_values)
    time = generate_time(total_values)
    cores = generate_cores(total_values)
    gpus = generate_gpus(total_values)
    nt = (noise/100)
    ni = np.random.uniform(0.01,nt/2)
    nbi = np.random.uniform(0.01,nt/2)
    n = [ni,nbi,nt]
    tasks = [list(t) for t in zip(inst, bi, time)]
    matched = create_dict(tasks,n)
    
    print ("DONE")
    if percentage is None:
        
        data = {
        'instructions': inst,
        'branch_instructions': bi,
        'time': time,
        'cpu_cycles': cpucy,
        'cache_misses': cache_misses,
        'branch_misses': bm,
        'memory': memory,
        'gpus': gpus,
        'cores': cores,
        'power': power
        }
    
        gd = pd.DataFrame(data)
        gd.to_parquet('generated_values.parquet', index=False)
        
        per_out = calc_percentage(matched)
        
    ##DATA_ADJUSTMENT:
    else:
        p = percentage
        regu = int(total_values * (percentage/100))
        irregu = int(total_values - regu)
        dist = aux(regu, irregu)

        matched= fit(matched,dist,n) ###UPDATE THE FITTING FUNCTION FOR THE DICTIONARY 
        per_out = calc_percentage(matched)
        ##matched = fix(matched, total_values,interval)
        ##per_out = calc_percentage(matched)
        
        lent = calc_values(matched)
        
        matched = add_label(matched)
        
        print (lent)
        print (matched)
        all_arrays = []
        for key, arrays in matched.items():
            for array in arrays:
                all_arrays.append(array)

        
        random.shuffle(all_arrays)
        df = pd.DataFrame(all_arrays, columns=['inst', 'bi', 'time', "label"])

        df.to_parquet('teste2.parquet', engine='pyarrow')
        ##data = {
        ##'instructions': inst,
        ##'cpu_cycles': cpucy,
        ##'cache_misses': cacmis,
        ##'branch_instructions': bi,
        ##'branch_misses': bm,
        ##'memory': memory,
        ##'gpus': gpus,
        ##'cores': cores,
        ##'time': time,
        ##'power': power
        ##}
    
        ##gd = pd.DataFrame(data)
        ##gd.to_parquet('generated_values.parquet', index=False)
    return per_out


In [31]:
print(generate_values(200000,10,50))

DONE


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



50.0


In [12]:
def create_dict(l, noise):
    check_tasks = l
    matched_dict = {}
    i = 0  # Initialize i
    n1, n2, n3 = noise  # Unpack noise values
    while i < len(check_tasks):
        matched_list = [check_tasks[i]]  # Initialize matched_list with the current index i
        value = check_tasks[i]
        indices_to_remove = []

        for j in range(i + 1, len(check_tasks)):
            if (abs(check_tasks[j][0] - value[0]) <= n1 * value[0] and
                    abs(check_tasks[j][1] - value[1]) <= n2 * value[1] and
                    abs(check_tasks[j][2] - value[2]) <= n3 * value[2]) and (i != j):
                matched_list.append(check_tasks[j])
                indices_to_remove.append(j)

        for idx in sorted(indices_to_remove, reverse=True):
            del check_tasks[idx]

        matched_dict[i] = matched_list
        i += 1

    sorted_matched_dict = {k: v for k, v in sorted(matched_dict.items(), key=lambda item: len(item[1]), reverse=True)}
    return sorted_matched_dict



In [13]:
def calc_values(matched_dic):
    lent = sum(len(matched_list) for matched_list in matched_dic.values())
    
    return lent

In [14]:
def calc_percentage (matched):
    total = 0
    reg = 0

    for key, value in matched.items():
        total += len(value)
        if len(value) > 1:
            reg += len(value)

    per = ((reg / total) * 100)
    
    return per

In [15]:
def check_percentage_individual(l, num_values, noise):
    matched = []
    i = 0  

    while i < len(l):
        matched_list = [l[i]]  
        value = l[i]
        indices_to_remove = []  
        for j in range(i + 1, len(l)):  
            if (abs(l[j] - value) <= noise * value):
                matched_list.append(l[j])
                indices_to_remove.append(j)
                
        for idx in sorted(indices_to_remove, reverse=True):
            del l[idx]
        
        if len(matched_list) > 0:
            matched.append(matched_list)  
        i += 1

    total = 0
    reg = 0

    for value in matched:
        total += len(value)
        if len(value) > 1:
            reg += len(value)
    
    per = ((reg / total) * 100)         
    
    return matched, per


### Generate Power:

In [16]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
def load_data(path: str) -> pd.DataFrame:
    """Load the data from a parquet file.

    Args:
        path (str): Path to the parquet file.

    Returns:
        pd.DataFrame: Loaded DataFrame.
    """
    return pd.read_parquet(path)

if __name__ == "__main__":
    
    sns.set_theme(style="whitegrid")

    DATA_PATH = "C://Users/Nuno/job_table.parquet"

    df = load_data(DATA_PATH)

    df_completed = df[df['job_state'] == 'COMPLETED'].copy()  # Use copy() to explicitly create a copy

    df_completed['job_state'].replace({"OUT_OF_MEMORY": "OOM+NODE FAIL", "NODE_FAIL": "OOM+NODE FAIL"}, inplace=True)


In [17]:

import pandas as pd
import math

def generate_power(num_values):
    lis = df_completed["power_consumption"]
    total_power = []

    for i in range(len(lis)):
        mean = np.mean(lis.iloc[i])
        total_power.append(mean)

    counts_pow, bin_edges_pow = np.histogram(total_power, bins='auto')

    percentages_pow = [(count / len(total_power)) * 100 for count in counts_pow]

    total_counts = num_values

    counts_pow = [(p * total_counts / 100) for p in percentages_pow]

    min_counts = [1 if count == 0 and total_counts > 0 else count for count in counts_pow]

    total_counts -= sum(counts_pow) - sum(min_counts)

    percentages_pow = [(count / sum(min_counts)) * 100 for count in min_counts]

    pow1 = []

    for count, (start, end) in zip(min_counts, zip(bin_edges_pow[:-1], bin_edges_pow[1:])):
        values = np.random.randint(start, end, math.ceil(count))
        pow1.extend(values)
    
    np.random.shuffle(pow1)


    return pow1[:num_values]


### Generate relations:


In [18]:
import numpy as np
import pandas as pd
import math

def generate_instructions(num_values):
    power = pd.read_csv('C://Users/Nuno/HPC-ODA/power_prediction/responses/responses.csv', usecols=[1], header=0, names=['y'])['y'].values
    instructions = pd.read_csv('C://Users/Nuno/HPC-ODA/power_prediction/sensors/node0.instructions.csv', usecols=[1], header=0, names=['y'])['y'].values[1:-1]

    inspow = np.divide(instructions, power)

    counts_ins, bin_edges_ins = np.histogram(inspow, bins='auto')

    # Calculate percentages
    percentages_inst = [(count / len(inspow)) * 100 for count in counts_ins]

    # Calculate the counts based on the percentages
    total_counts = num_values

    counts_inst = [(p * total_counts / 100) for p in percentages_inst]

    # Ensure that each bin contributes at least one value
    min_counts = [1 if count == 0 and total_counts > 0 else count for count in counts_inst]

    # Adjust total_counts
    total_counts -= sum(counts_inst) - sum(min_counts)

    # Recalculate percentages
    percentages_inst = [(count / sum(min_counts)) * 100 for count in min_counts]

    # Generate random values based on intervals and counts
    inspow1 = []

    for count, (start, end) in zip(min_counts, zip(bin_edges_ins[:-1], bin_edges_ins[1:])):
        # Generate random values within each bin
        values = np.random.uniform(start, end, math.ceil(count))
        inspow1.extend(values)

    # Shuffle the generated values to make them random
    np.random.shuffle(inspow1)
    
    inspow1 = inspow1[:num_values]
    power = generate_power(num_values)
    inst = [round(inspow1[i] * power[i]) for i in range(num_values)]

    return inst  # Trim to the requested number of values


In [19]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


import numpy as np
import pandas as pd
import math

def generate_cache_misses(num_values):
    power = pd.read_csv('C://Users/Nuno/HPC-ODA/power_prediction/responses/responses.csv', usecols=[1], header=0, names=['y'])['y'].values
    cache_misses = pd.read_csv('C://Users/Nuno/HPC-ODA/power_prediction/sensors/node0.cache-misses.csv', usecols=[1], header=0, names=['y'])['y'].values[1:-1]

    cachepow = np.divide(cache_misses, power)

    counts_cache, bin_edges_cache = np.histogram(cachepow, bins='auto')

    # Calculate percentages
    percentages_cache = [(count / len(cachepow)) * 100 for count in counts_cache]

    # Calculate the counts based on the percentages
    total_counts = num_values

    counts_cache = [(p * total_counts / 100) for p in percentages_cache]

    # Ensure that each bin contributes at least one value
    min_counts = [1 if count == 0 and total_counts > 0 else count for count in counts_cache]

    # Adjust total_counts
    total_counts -= sum(counts_cache) - sum(min_counts)

    # Recalculate percentages
    percentages_cache = [(count / sum(min_counts)) * 100 for count in min_counts]

    # Generate random values based on intervals and counts
    cachepow1 = []

    for count, (start, end) in zip(min_counts, zip(bin_edges_cache[:-1], bin_edges_cache[1:])):
        # Generate random values within each bin
        values = np.random.uniform(start, end, math.ceil(count))
        cachepow1.extend(values)

    # Shuffle the generated values to make them random
    np.random.shuffle(cachepow1)
    
    cachepow1[:num_values]
    power = generate_power(num_values)
    cm = [round(cachepow1[i] * power[i]) for i in range(num_values)]

    return cm # Trim to the requested number of values


In [20]:

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

def generate_cpu_cycles(num_values):
    power = pd.read_csv('C://Users/Nuno/HPC-ODA/power_prediction/responses/responses.csv', usecols=[1], header=0, names=['y'])['y'].values
    cpu_cycles = pd.read_csv('C://Users/Nuno/HPC-ODA/power_prediction/sensors/node0.cpu-cycles.csv', usecols=[1], header=0, names=['y'])['y'].values[1:-1]

    cpupow = np.divide(cpu_cycles, power)

    counts_cpu, bin_edges_cpu = np.histogram(cpupow, bins='auto')

    # Calculate percentages
    percentages_cpu = [(count / len(cpupow)) * 100 for count in counts_cpu]

    # Calculate the counts based on the percentages
    total_counts = num_values

    counts_cpu = [(p * total_counts / 100) for p in percentages_cpu]

    # Ensure that each bin contributes at least one value
    min_counts = [1 if count == 0 and total_counts > 0 else count for count in counts_cpu]

    # Adjust total_counts
    total_counts -= sum(counts_cpu) - sum(min_counts)

    # Recalculate percentages
    percentages_cpu = [(count / sum(min_counts)) * 100 for count in min_counts]

    # Generate random values based on intervals and counts
    cpupow1 = []

    for count, (start, end) in zip(min_counts, zip(bin_edges_cpu[:-1], bin_edges_cpu[1:])):
        # Generate random values within each bin
        values = np.random.uniform(start, end, math.ceil(count))
        cpupow1.extend(values)

    # Shuffle the generated values to make them random
    np.random.shuffle(cpupow1)
    
    cpupow1[:num_values]
    power = generate_power(num_values)
    cpp= [round(cpupow1[i] * power[i]) for i in range(num_values)]

    return cpp  # Trim to the requested number of values


In [21]:

import numpy as np
import pandas as pd
import math

def generate_branch_instructions(num_values):
    power = pd.read_csv('C://Users/Nuno/HPC-ODA/power_prediction/responses/responses.csv', usecols=[1], header=0, names=['y'])['y'].values
    branch_instructions = pd.read_csv('C://Users/Nuno/HPC-ODA/power_prediction/sensors/node0.branch-instructions.csv', usecols=[1], header=0, names=['y'])['y'].values[1:-1]

    bipow = np.divide(branch_instructions, power)

    counts_bi, bin_edges_bi = np.histogram(bipow, bins='auto')

    # Calculate percentages
    percentages_bi = [(count / len(bipow)) * 100 for count in counts_bi]

    # Calculate the counts based on the percentages
    total_counts = num_values

    counts_bi = [(p * total_counts / 100) for p in percentages_bi]

    # Ensure that each bin contributes at least one value
    min_counts = [1 if count == 0 and total_counts > 0 else count for count in counts_bi]

    # Adjust total_counts
    total_counts -= sum(counts_bi) - sum(min_counts)

    # Recalculate percentages
    percentages_bi = [(count / sum(min_counts)) * 100 for count in min_counts]

    # Generate random values based on intervals and counts
    bipow1 = []

    for count, (start, end) in zip(min_counts, zip(bin_edges_bi[:-1], bin_edges_bi[1:])):
        # Generate random values within each bin
        values = np.random.uniform(start, end, math.ceil(count))
        bipow1.extend(values)

    # Shuffle the generated values to make them random
    np.random.shuffle(bipow1)
    bipow1 = bipow1[:num_values]
    power = generate_power(num_values)
    bi = [round(bipow1 [i] * power[i]) for i in range(num_values)]

    return bi  # Trim to the requested number of values


In [22]:
import numpy as np
import pandas as pd
import math

def generate_branch_misses(num_values):
    power = pd.read_csv('C://Users/Nuno/HPC-ODA/power_prediction/responses/responses.csv', usecols=[1], header=0, names=['y'])['y'].values
    branch_misses = pd.read_csv('C://Users/Nuno/HPC-ODA/power_prediction/sensors/node0.branch-instructions.csv', usecols=[1], header=0, names=['y'])['y'].values[1:-1]

    bmpow = np.divide(branch_misses, power)

    counts_bm, bin_edges_bm = np.histogram(bmpow, bins='auto')

    # Calculate percentages
    percentages_bm = [(count / len(bmpow)) * 100 for count in counts_bm]

    # Calculate the counts based on the percentages
    total_counts = num_values

    counts_bm = [(p * total_counts / 100) for p in percentages_bm]

    # Ensure that each bin contributes at least one value
    min_counts = [1 if count == 0 and total_counts > 0 else count for count in counts_bm]

    # Adjust total_counts
    total_counts -= sum(counts_bm) - sum(min_counts)

    # Recalculate percentages
    percentages_bm = [(count / sum(min_counts)) * 100 for count in min_counts]

    # Generate random values based on intervals and counts
    bmpow1 = []

    for count, (start, end) in zip(min_counts, zip(bin_edges_bm[:-1], bin_edges_bm[1:])):
        # Generate random values within each bin
        values = np.random.uniform(start, end, math.ceil(count))
        bmpow1.extend(values)

    # Shuffle the generated values to make them random
    np.random.shuffle(bmpow1)
    
    bmpow1[:num_values]
    power = generate_power(num_values)
    bmp= [round(bmpow1[i] * power[i]) for i in range(num_values)]

    # Trim to the requested number of values
    return bmp


### Generate Mixed Data:

In [23]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import math

def generate_time(num_values):
    time = np.array(df_completed["run_time"])
    
    counts_time, bin_edges_time = np.histogram(time, bins='auto')

    percentages_time = [(count / len(time)) * 100 for count in counts_time]

    # Calculate the counts based on the percentages
    total_counts = num_values

    counts_time = [(p * total_counts / 100) for p in percentages_time]

    # Ensure that each bin contributes at least one value
    min_counts = [1 if count == 0 and total_counts > 0 else count for count in counts_time]

    # Adjust total_counts
    total_counts -= sum(counts_time) - sum(min_counts)

    # Recalculate percentages
    percentages_time = [(count / sum(min_counts)) * 100 for count in min_counts]

    # Generate random values based on intervals and counts
    time1 = []

    for count, (start, end) in zip(min_counts, zip(bin_edges_time[:-1], bin_edges_time[1:])):
        # Generate random values within each bin
        values = np.random.randint(start, end, math.ceil(count))
        time1.extend(values)

    # Shuffle the generated values to make them random
    np.random.shuffle(time1)

    return time1[:num_values]  # Trim to the requested number of values


In [24]:
import numpy as np
import pandas as pd
import math

def generate_memory(num_values):
    mem = np.array(df_completed["mem_alloc"])
    
    counts_mem, bin_edges_mem = np.histogram(mem, bins='auto')

    # Calculate percentages
    percentages_mem = [(count / len(mem)) * 100 for count in counts_mem]

    # Calculate the counts based on the percentages
    total_counts = num_values

    counts_mem = [(p * total_counts / 100) for p in percentages_mem]

    # Ensure that each bin contributes at least one value
    while sum(counts_mem) < num_values:
        counts_mem = [count + 1 for count in counts_mem]

    # Generate random values based on intervals and counts
    mem1 = []

    for count, (start, end) in zip(counts_mem, zip(bin_edges_mem[:-1], bin_edges_mem[1:])):
        # Generate random values within each bin
        values = np.random.randint(start, end, math.ceil(count))
        mem1.extend(values)

    # Shuffle the generated values to make them random
    np.random.shuffle(mem1)

    return mem1[:num_values]  # Trim to the requested number of values


In [25]:
import numpy as np
import pandas as pd
import math

def generate_memory1(num_values):
    mem = np.array(df_completed["mem_alloc"])
    
    counts_mem, bin_edges_mem = np.histogram(mem, bins='auto')
    
    percentages_mem = [(count / len(mem)) * 100 for count in counts_mem]
    
    
    total_counts = num_values
    
    mem1 = []
    i = 0

    while (len(mem1) < num_values):
        print (len(counts))
        print (len(mem1))
        if i >= total_counts:
            for j in range(len(counts)):
                percentages_mem[j] = (counts[j] / (sum(counts))) * 100
            i = 0
        counts = [(p * total_counts / 100) for p in percentages_mem]
        for count, (start, end) in zip(counts, zip(bin_edges_mem[:-1], bin_edges_mem[1:])):
            if count < 0:
                count =0 
            values = np.random.randint(start, end, int(count))
            mem1.extend(values)
            total_counts = total_counts - int(count)
            count = count - (int(count))
            i = i + 1
            
    np.random.shuffle(mem1)
    
    return mem1[:num_values]



In [26]:
import numpy as np
import pandas as pd

def generate_gpus(num_values):
    gpus = np.array(df_completed["num_gpus_alloc"])
    total_counts = num_values
    count_dict = {}
    for num in gpus:
        count_dict[num] = count_dict.get(num, 0) + 1

    percentages_gpus = [(count / len(gpus)) * 100 for count in count_dict.values()]
    counts_gpus = [int(p * total_counts / 100) for p in percentages_gpus]
    
    while sum(counts_gpus) < num_values:
        counts_gpus = [count + 1 for count in counts_gpus]
    
    i = 0
    gpus1 = []
    for num, count in count_dict.items():
        gpus1.extend([num] * counts_gpus[i])
        i = i + 1

    np.random.shuffle(gpus1)

    return gpus1[:num_values]



In [27]:
import numpy as np
import pandas as pd

def generate_cores(num_values):
    cores = np.array(df_completed["num_cores_alloc"])
    total_counts = num_values
    count_dict = {}
    for num in cores:
        count_dict[num] = count_dict.get(num, 0) + 1

    percentages_cores = [(count / len(cores)) * 100 for count in count_dict.values()]
    counts_cores = [int(p * total_counts / 100) for p in percentages_cores]
    
    while sum(counts_cores) < num_values:
        counts_cores = [count + 1 for count in counts_cores]

    cores1 = []
    i = 0
    for num, count in count_dict.items():
        cores1.extend([num] * counts_cores[i])
        i = i + 1

    np.random.shuffle(cores1)

    return cores1[:num_values]


### How is this data generated:

The power consumption, execution time, number of cores allocated, number of gpus allocated and memory allocated result from the distribution of data in the PM100 dataset which comes from the M100 Exadata. The data was analysed and the building the histograms of each of these parameters we extracted the probability of data points in certain intervals considering that these distributions did not come close to fit a known distribution.

Using the HPC-ODA dataset, we extracted the relations (with the same steps as before) between instructions and power, cache misses and power, branch instructions and power, branch misses and power and cpu cycles and power with these distributions we generated random relations within it and multipled it by the value of power generated above.

### Adding tasks that would be recognized as regular/periodic tasks

In order to do this we use a task that is generated randomly, copy it and spread it a number of times through the dataset. This can be done for one or more tasks and the copy will not be exacly alike but instead just identical in order to recreate in a syntethical way a real regular enviroment of task. Still undecided on the criticallity of tasks and dependecies of tasks.